# IZV - 3. PROJEKT - 2. ÚKOL - Test Hypotézy
## Autor: Ladislav Vašina (xvasin11)
### Hypotéza 1
*Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako nasilnicích třetí třídy.*

Nejdříve si naimportujeme potřebné knihovny

In [110]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind

Načteme si náš dataset

In [111]:
df_orig = pd.read_pickle('accidents.pkl.gz')
df = df_orig.copy()

Vyfiltrujeme dataset tak, aby obsahoval pouze nehody, které se staly na silnicích 1. a 3. třídy.
Také si vytvoříme pomocný sloupec značící zda při nehodě někdo zahynul.

In [112]:
df = df[df['p36'].isin([1, 3])]
df['death'] = df['p13a'] > 0

Teď si vytvoříme kontingenční tabulku. Ta se využívá k vizualizaci vztahu nějakých statistických znaků.

In [113]:
contingency_table = pd.crosstab(df['death'], df['p36'])
contingency_table

p36,1,3
death,,
False,78618,73352
True,911,448


Provedeme výpočet $\chi^2$ s využitím funkce *chi2_contingency*

In [114]:
result = chi2_contingency(contingency_table) # returns stat, p, dof, expected
result

(125.72070150000258,
 3.5395243450138555e-29,
 1,
 array([[78824.11109444, 73145.88890556],
        [  704.88890556,   654.11109444]]))

Nyní si zjistíme, na kterém druhu silnic dochází častěji k fatální nehodě

In [115]:
more_dangerous_road = contingency_table - result[3]
more_dangerous_road

p36,1,3
death,,
False,-206.111094,206.111094
True,206.111094,-206.111094


### Závěr 1. hypotézy
P hodnota nám vyšla $3,54 \cdot 10^{-29}$.
Tato hodnodnota je menší než $0,05$, z čehož můžeme rozhodnout, že nulová hypotéza neplatí.
Z rozdílu očekávaných a získaných výsledků můžeme určit, že se na silnicích 1. třídy umíralo častěji než na silnicíh 3. třídy.


## Hypotéza 2
*Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.*

Vyfiltrujeme nehody tak, aby obsahovaly pouze nehody, kde figurovalo auto značky Škoda či Audi

In [116]:
df2 = df_orig.copy()
df2 = df2[df2['p45a'].isin([2, 39])]

df2['p45a'] = df2['p45a'].replace({2: 'Audi', 39: 'Škoda'})
df2['p14'] = df2['p14'].astype(int)

skoda_group = df2.loc[df2['p45a'] == 'Škoda']
audi_group = df2.loc[df2['p45a'] == 'Audi']

skoda_group = skoda_group['p14']
audi_group = audi_group['p14']

Nyní máme vytvořené dvě skupiny nehod. Nehody vozidel Škoda a Audi.
Provedeme t test.

In [117]:
ttest_ind(a=audi_group, b=skoda_group, alternative="greater")

Ttest_indResult(statistic=23.391701716453692, pvalue=4.622063681663004e-121)

### Závěr 2. hypotézy
Použitím t testu s parametrem alternative='greater' nám vyšla P hodnota $4,622 \cdot 10^{-121}$ což nám říká, že se dvě porovnávané skupiny nerovnají a že první skupina (v tomto případě nehody vozidel Audi) měla způsobenou větší hmotnou škodu než skupina druhá (nehody vozidel Škoda).

Můžeme tedy říci, že Hypotéza 2 je pravdivá.